In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
tips = sns.load_dataset('tips')

In [4]:
num = tips._get_numeric_data().columns
cat = tips.select_dtypes('category').columns


In [5]:
from collections import defaultdict

data_dict1= defaultdict(list)

In [6]:
for x in num:
    for y in cat:
        data_dict1[x].append(y)

In [7]:
data_dict1

defaultdict(list,
            {'total_bill': ['sex', 'smoker', 'day', 'time'],
             'tip': ['sex', 'smoker', 'day', 'time'],
             'size': ['sex', 'smoker', 'day', 'time']})

In [8]:
di = {}
li = []
for x in cat:
    li.append(x)
    
for y in num:
    di.update({y:li})

In [9]:
li

['sex', 'smoker', 'day', 'time']

In [10]:
di

{'total_bill': ['sex', 'smoker', 'day', 'time'],
 'tip': ['sex', 'smoker', 'day', 'time'],
 'size': ['sex', 'smoker', 'day', 'time']}

### ---------------------------------------------- Source_1 ----------------------------------------------

In [11]:
np.random.seed(12)

races =   ["asian","black","hispanic","other","white"]

# Generate random data
voter_race = np.random.choice(a= races,
                              p = [0.05, 0.15 ,0.25, 0.05, 0.5],
                              size=1000)

voter_age = stats.poisson.rvs(loc=18,
                              mu=30,
                              size=1000)

# Group age data by race
voter_frame = pd.DataFrame({"race":voter_race,"age":voter_age})
groups = voter_frame.groupby("race").groups

voter_frame.head(2)

NameError: name 'stats' is not defined

In [184]:
voter_frame.groupby('race').agg(['mean','count'])

age      
               mean count
race                     
asian     48.886364    44
black     47.551020   147
hispanic  48.127049   244
other     48.860000    50
white     47.467961   515

In [162]:
asian

array([56, 52, 37, 50, 53, 47, 56, 43, 46, 54, 45, 54, 42, 44, 55, 50, 45,
       49, 51, 57, 56, 46, 43, 53, 48, 54, 54, 44, 40, 46, 51, 52, 44, 54,
       43, 44, 53, 42, 54, 44, 59, 47, 54, 40])

In [161]:
# Etract individual groups
asian = voter_age[groups["asian"]]
black = voter_age[groups["black"]]
hispanic = voter_age[groups["hispanic"]]
other = voter_age[groups["other"]]
white = voter_age[groups["white"]]

# Perform the ANOVA
stats.f_oneway(asian, black, hispanic, other, white)


F_onewayResult(statistic=1.7744689357329695, pvalue=0.13173183201930463)

### ----------------------------------------- Source_2 ------------------------------------------------

https://pythonfordatascience.org/anova-python/

### ------------------------------------- Source_3 ----------------------------------------------------

In [10]:
data = pd.read_csv('/home/rohan/Desktop/jupyter/ROHAN_ALL/SM_&_ML/Datasets/PlantGrowth.csv')
data.head(2)

,Unnamed: 0,weight,group
0,1,4.17,ctrl
1,2,5.58,ctrl


In [16]:
grps = data.group.unique()

In [17]:
data.shape

(30, 3)

#### distribution based on category "group"

In [18]:
d_data = {grp:list(data.loc[data.group == grp,'weight']) for grp in grps}
pd.DataFrame(d_data)

,ctrl,trt1,trt2
0,4.17,4.81,6.31
1,5.58,4.17,5.12
2,5.18,4.41,5.54
3,6.11,3.59,5.50
4,4.50,5.87,5.37
5,4.61,3.83,5.29
6,5.17,6.03,4.92
7,4.53,4.89,6.15
8,5.33,4.32,5.80
9,5.14,4.69,5.26


In [19]:
k = len(pd.unique(data.group))  # number of conditions
N = len(data.values)  # conditions times participants
n = data.groupby('group').size()[0] #Participants in each condition

print('number of conditions:',k,'|| conditions times participants:',N,'|| participants in each condition:',n)

number of conditions: 3 || conditions times participants: 30 || participants in each condition: 10


In [20]:
d_data['ctrl']

[4.17, 5.58, 5.18, 6.11, 4.5, 4.61, 5.17, 4.53, 5.33, 5.14]

In [21]:
from scipy import stats

F, p = stats.f_oneway(d_data['ctrl'], d_data['trt1'], d_data['trt2'])

In [22]:
print('f_stats:',F,'|| P_value:',p)

f_stats: 4.846087862380136 || P_value: 0.0159099583256229


One problem with using SciPy is that  following APA guidelines we should also effect size (e.g., eta squared) as well as Degree of freedom (DF). DFs needed for the example data is easily obtained

In [23]:
DFbetween = k - 1
DFwithin = N - k
DFtotal = N - 1

#### Using statsmodel.Api

In [24]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

mod = ols('weight ~ group',
                data=data).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
group,3.76634,2.0,4.846088,0.01591
Residual,10.49209,27.0,NaN,NaN


In [25]:
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 weight   R-squared:                       0.264
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     4.846
Date:                Sat, 16 Feb 2019   Prob (F-statistic):             0.0159
Time:                        08:58:23   Log-Likelihood:                -26.810
No. Observations:                  30   AIC:                             59.62
Df Residuals:                      27   BIC:                             63.82
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         5.0320      0.197     25.527      0.000       4.628       5.436
group[T.trt1]    -0.3710      0.279     -1.331      0.194      -0.943       0.201
group[T.trt2]     0.4940      0.279      1.772      0.088      -0.078       1.066
==============================================================================
Omnibus:                        1.835   Durbin-Watson:                   2.704
Prob(Omnibus):                  0.400   Jarque-Bera (JB):                1.406
Skew:                           0.524   Prob(JB):                        0.495
Kurtosis:                       2.835   Cond. No.                         3.73
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### -----------------------------------------------

In [11]:
tips = sns.load_dataset('tips')

In [12]:
tips.head(2)

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3


In [13]:
tips.groupby('sex').agg(['mean','min','max'])

total_bill                    tip                 size        
             mean   min    max      mean  min   max      mean min max
sex                                                                  
Male    20.744076  7.25  50.81  3.089618  1.0  10.0  2.630573   1   6
Female  18.056897  3.07  44.30  2.833448  1.0   6.5  2.459770   1   6

In [14]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

mod = ols('tip ~ sex',
                data=tips).fit()
                
aov_table = sm.stats.anova_lm(mod, typ=2)
aov_table

,sum_sq,df,F,PR(>F)
sex,3.673534,1.0,1.926155,0.166456
Residual,461.538943,242.0,NaN,NaN


### Anova for each Num_Cat Pair

#### Appending Dataframe directly

#### Appending

In [15]:
def Anova_looper(df):
    num = df._get_numeric_data().columns
    cat = df.select_dtypes(['object','category']).columns

    li = []
    lt = []
    for x in num:
        for y in cat:
            lt.append([x,y])
            dat = ols('{} ~ {}'.format(x,y),
                    data=df).fit()

            aov_table = sm.stats.anova_lm(dat, typ=2)
            result = aov_table[['F','PR(>F)']].iloc[0].tolist()
            li.append(result)

    # ---------------------------------

    key =pd.DataFrame(lt,columns=['DV_Num','IV_Cat'])
    key.head(2)

    value =pd.DataFrame(li,columns=['F_value','P_value'])
    value.head(2)

    df = pd.merge(key,value,left_index=True,right_index=True)
    df['relation'] = np.where(df.P_value<0.1,'Corelated','Not_Corelated')
    return df

In [17]:
Anova_looper(tips)

,DV_Num,IV_Cat,F_value,P_value,relation
0,total_bill,sex,5.188346,0.023612,Corelated
1,total_bill,smoker,1.791412,0.182010,Not_Corelated
2,total_bill,day,2.767479,0.042454,Corelated
3,total_bill,time,8.396303,0.004105,Corelated
4,tip,sex,1.926155,0.166456,Not_Corelated
5,tip,smoker,0.008506,0.926593,Not_Corelated
6,tip,day,1.672355,0.173589,Not_Corelated
7,tip,time,3.633815,0.057802,Corelated
8,size,sex,1.811408,0.179599,Not_Corelated
9,size,smoker,4.369723,0.037627,Corelated
